In [1]:
import pandas as pd
import numpy as np
from openbb import obb

In [2]:
def _fetch_ohlc_openbb(symbol: str, start_date: str = "2010-01-01", end_date: str = None, interval: str = "1W",
                       provider: str = "yfinance") -> pd.DataFrame:
    """
    从 OpenBB 获取标的历史日线 OHLC 数据，转为统一格式 DataFrame。
    列名统一为: Open, High, Low, Close, Adj Close（无 adj_close 时用 close）。
    """
    result = obb.equity.price.historical(
        symbol=symbol,
        start_date=start_date,
        end_date=end_date,
        interval=interval,
        provider=provider,
    )
    df = result.to_df()
    if df is None or df.empty:
        raise ValueError(f"OpenBB 未返回数据: {symbol}")
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)
    col_map = {"close": "Close"}
    df = df.rename(columns=col_map)
    if "adj_close" in df.columns:
        df["Close"] = df["adj_close"]
    else:
        df["Close"] = df["Close"]
    df = df[["Close"]].copy()
    df = df.sort_index(ascending=False)
    df["Return"] = df["Close"] / df["Close"].shift(-1) - 1
    return df

In [6]:
# 投资组合标的字典 {ticker: 名称}
PORTFOLIO = {
    "V": "Visa", "DIS": "Disney", "NKE": "Nike", "ORCL": "Oracle", "CAT": "Caterpillar",
    "META": "Meta", "AAPL": "Apple", "AMZN": "Amazon", "NFLX": "Netflix", "MSFT": "Microsoft", "GOOG": "Alphabet",
    "HSY": "Hershey", "STZ": "Constellation", "ALGN": "Align", "XEL": "Xcel Energy", "PPL": "PPL",
    "MAR": "Marriott", "LVS": "Las Vegas Sands", "LEN": "Lennar", "BBY": "Best Buy", "LUV": "Southwest",
    "WING": "Wingstop", "FRPT": "Freshpet", "PEGA": "Pegasystems", "OLED": "Universal Display",
    "XPO": "XPO Logistics", "TREX": "Trex", "MOS": "Mosaic", "OLLI": "Ollie's", "DKS": "Dick's Sporting",
     "VIRT": "Virtu", "JBLU": "JetBlue",
}

In [4]:
_fetch_ohlc_openbb(symbol="SPY", interval="1W")

,Close,Return
date,,
2026-01-30,677.619995,-0.023659
2026-01-23,694.039978,0.007344
2026-01-16,688.979980,-0.004709
2026-01-09,692.239990,0.003959
2026-01-02,689.510010,0.011130
...,...,...
2010-01-29,106.440002,-0.019619
2010-01-22,108.570000,-0.028021
2010-01-15,111.699997,-0.028104


In [ ]:
# 遍历所有标的，生成 returns_df：列=标的，index=date，值=当日 Return
returns_list = []
for ticker in PORTFOLIO:
    try:
        df = _fetch_ohlc_openbb(symbol=ticker, start_date = "2015-07-31", interval = "1W")
        returns_list.append(df[["Return"]].rename(columns={"Return": ticker}))
    except Exception as e:
        print(f"{ticker} 失败: {e}")

returns_df = pd.concat(returns_list, axis=1)
returns_df = returns_df.dropna(how="all")  # 删除全空行
# returns_df = returns_df.dropna()  # 可选：仅保留所有标均有数据的日期

,V,DIS,NKE,ORCL,CAT,META,AAPL,AMZN,NFLX,MSFT,...,FRPT,PEGA,OLED,XPO,TREX,MOS,OLLI,DKS,VIRT,JBLU
date,,,,,,,,,,,,,,,,,,,,,
2026-02-02,0.022683,-0.069415,0.014399,-0.170738,0.031870,-0.064606,0.063319,-0.069411,-0.031381,-0.085105,...,-0.033429,-0.125658,-0.015415,0.253258,0.038629,0.009455,-0.032635,0.017475,-0.079499,0.240246
2026-01-26,-0.013336,0.016399,-0.049662,-0.071009,0.049057,0.087650,0.046122,0.000585,-0.030539,-0.076532,...,-0.025856,-0.132791,-0.043963,0.005704,-0.026557,-0.044807,-0.024927,-0.020179,0.117061,-0.077652
2026-01-19,-0.006457,-0.001978,0.010252,-0.072898,-0.031335,0.062088,-0.029312,0.000167,-0.021364,0.013243,...,0.113619,-0.030968,0.032585,-0.020876,-0.026984,0.092600,-0.029843,-0.042541,0.007319,-0.018587
2026-01-12,-0.061383,-0.040387,-0.023362,-0.037427,0.047392,-0.050240,-0.014805,-0.033390,-0.016320,-0.040519,...,0.002966,-0.102692,-0.075217,0.025080,0.049940,0.014242,-0.015866,-0.004024,0.113156,0.036609
2026-01-05,0.009495,0.036030,0.041719,0.014358,0.032102,0.004074,-0.042950,0.092185,-0.016815,0.013405,...,0.065004,0.033535,0.032171,0.057209,0.162434,0.038369,0.064409,0.079762,0.016564,0.130719


In [11]:
df_corr = returns_df.corr()
df_cov = returns_df.cov()

In [12]:
df_corr.to_csv("./export/PortFolioCorr.csv")
df_cov.to_csv("./export/PortFolioCov.csv")
